# IF3170 Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 20

Group Members:
- Ahmad Naufal Ramadan (13522005)
- Kristo Anugrah (13522024)
- Tazkia Nizami (13522032)
- Farhan Nafis Rayhan (13522037)

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import pickle


# Import other libraries if needed

## Import Dataset

In [ ]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public

# df = pd.read_csv('https://drive.google.com/uc?id=1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU')
# df.head()

# Membaca semua file csv training
additional_features_df = pd.read_csv('../dataset/train/additional_features_train.csv')
basic_features_df = pd.read_csv('../dataset/train/basic_features_train.csv')
content_features_df = pd.read_csv('../dataset/train/content_features_train.csv')
flow_features_df = pd.read_csv('../dataset/train/flow_features_train.csv')
labels_df = pd.read_csv('../dataset/train/labels_train.csv')
time_features_df = pd.read_csv('../dataset/train/time_features_train.csv')

# Menggabungkan data training dan testing untuk analisis EDA menyeluruh
data_with_label = pd.merge(basic_features_df, additional_features_df, on="id")
data_with_label = pd.merge(data_with_label, content_features_df, on="id")
data_with_label = pd.merge(data_with_label, flow_features_df, on="id")
data_with_label = pd.merge(data_with_label, time_features_df, on="id")
data_with_label = pd.merge(data_with_label, labels_df, on="id")
data = data_with_label.drop('label', axis=1)

data.head()

# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [ ]:
# Write your code here
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
binary_features = ['is_sm_ips_ports', 'is_ftp_login']

numerical_features = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
for feature in binary_features:
    numerical_features.remove(feature)
numerical_features.remove('id')

markdown_content = "### Total Features\n"
markdown_content += f"* Total features: {len(categorical_features) + len(numerical_features) + len(binary_features)}\n"
markdown_content += f"* Total categorical features: {len(categorical_features)}\n"
markdown_content += f"* Total numerical features: {len(numerical_features)}\n"
markdown_content += f"* Total binary features: {len(binary_features)}\n"
markdown_content += "\n### Categorical Features\n"
markdown_content += "\n".join([f"* {feature}" for feature in categorical_features])
markdown_content += "\n\n### Numerical Features\n"
markdown_content += "\n".join([f"* {feature}" for feature in numerical_features])
markdown_content += "\n\n### Binary Features\n"
markdown_content += "\n".join([f"* {feature}" for feature in binary_features])

display(Markdown(markdown_content))

In [ ]:
markdown_content = "### Categorical Features Values\n"
for feature in categorical_features:
    markdown_content += f"\n#### {feature}\n"
    markdown_content += f"* Unique values: {data[feature].nunique()}\n"
    markdown_content += f"* Values: {data[feature].unique()}\n"

display(Markdown(markdown_content))

markdown_content = "### Numerical Features Values\n"
for feature in numerical_features:
    markdown_content += f"\n#### {feature}\n"
    unique_values = data[feature].nunique()
    if unique_values > 150:
        markdown_content += f"* Min: {data[feature].min()}\n"
        markdown_content += f"* Mean: {data[feature].mean()}\n"
        markdown_content += f"* Max: {data[feature].max()}\n"
        markdown_content += f"* STD: {data[feature].std()}\n"
        markdown_content += f"* 25%: {data[feature].quantile(0.25)}\n"
        markdown_content += f"* 50%: {data[feature].quantile(0.50)}\n"
        markdown_content += f"* 75%: {data[feature].quantile(0.75)}\n"
    else:
        markdown_content += f"* Unique values: {unique_values}\n"
        markdown_content += f"* Values: {data[feature].unique()}\n"
    

display(Markdown(markdown_content))

In [ ]:
rows, columns = data.shape
display(Markdown(f"### Data shape:"))
display(Markdown(f"#### Number of Instances is: {rows}"))
display(Markdown(f"#### Number of Features is: {columns}"))

In [ ]:
categorical_values = data.select_dtypes(include=['object'])
binary_values = data.loc[:, binary_features]
numerical_values = data.select_dtypes(include=['int64', 'float64']) \
                            .drop(binary_features, axis=1).drop('id', axis=1)

display(Markdown("### Numerical values:"))
display(numerical_values)
display(Markdown("### Categorical values:"))
display(categorical_values)
display(Markdown("### Binary values:"))
display(binary_values)

In [ ]:
display(Markdown("### Numerical value statistics:"))
display(numerical_values.describe())

In [ ]:
missing_values = data.isnull().sum().to_frame("missing_values_count")
display(Markdown("### Number of missing values in every column:"))
display(missing_values)

In [ ]:
def histogram_numerical(numerical_table: pd.DataFrame):
  fig, axes = plt.subplots(9, 4, figsize=(30, 30))
  axes = axes.flatten()
  for index, col in enumerate(numerical_table.columns):
    axes[index].hist(numerical_table[col].dropna(), color='skyblue', edgecolor='black')
    axes[index].set_title(f'Distribution of {col}')
  fig.tight_layout()
    
display(Markdown("### Histogram of every numerical features:"))
histogram_numerical(numerical_values)

In [ ]:
def histogram_categorical(categorical_table: pd.DataFrame):
  fig, axes = plt.subplots(4, 1, figsize=(100, 100))
  axes = axes.flatten()
  for index, col in enumerate(categorical_table.columns):
    categorical_table[col].value_counts().plot(kind="bar", ax=axes[index]).set_title(col)
    #axes[index].hist(categorical_table[col].dropna(), color='skyblue', edgecolor='black')
    #axes[index].set_title(f'Distribution of {col}')
  fig.tight_layout()
    
display(Markdown("### Histogram of every categorical features:"))
histogram_categorical(categorical_values)

In [ ]:
# boxplot with outliers to see outliers in every feature

def boxplot_all_column(numerical_table: pd.core.frame.DataFrame):
  fig, axes = plt.subplots(9, 4, figsize = (20, 20))
  axes = axes.flatten() 
  for index, col in enumerate(numerical_table.columns):
    axes[index].boxplot(numerical_table[col].dropna(), False, sym="rs", vert=False,  widths=0.5, positions=[0])
    axes[index].set_title(f"Boxplot of {col}")
  fig.tight_layout()

display(Markdown("### Boxplot of every numerical features:"))
boxplot_all_column(numerical_values)

In [ ]:
def count_of_outliers(numerical_table: pd.DataFrame):
  Q1 = numerical_table.quantile(0.25)
  Q3 = numerical_table.quantile(0.75)
  IQR = Q3 - Q1
  return (((numerical_table < (Q1 - 1.5 * IQR)) | (numerical_table > (Q3 + 1.5 * IQR))).sum().to_frame("Number of outliers"))

count_of_outliers(numerical_values)

In [ ]:
import seaborn as sn

corr_numerical_matrix = numerical_values.corr()

plt.figure(figsize=(180, 180))

sn.heatmap(corr_numerical_matrix, annot=True, cmap='coolwarm', fmt='.2f', annot_kws={"size": 42})
plt.xticks(fontsize=36)
plt.yticks(fontsize=36)

display(Markdown("### Correlation heatmap of every pair of numerical features"))
plt.show()

In [ ]:
corr_categorical_matrix = categorical_values.apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1)
display(Markdown("### Correlation heatmap of every pair of categorical features"))
display(sn.heatmap(corr_categorical_matrix, annot=True, cmap='coolwarm', fmt='.2f', annot_kws={"size": 10}))

In [ ]:
def display_contingency_table(categorical_table: pd.DataFrame):
  columns = categorical_table.columns
  for index1, col1 in enumerate(columns):
    for index2, col2 in enumerate(columns):
      if index2 >= index1:
        break
      first_feature = categorical_values.loc[:, col1]
      second_feature = categorical_values.loc[:, col2]
      contingency_table = pd.crosstab(first_feature, second_feature)
      display(Markdown(f"#### Contingency table of {col1} with {col2}:"))
      display(contingency_table)

display_contingency_table(categorical_values)

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [ ]:
import sklearn.model_selection

X = data.drop(['attack_cat'], axis=1)
y = data['attack_cat']

X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=42)

display(Markdown("### X train:"))
display(X_train)
display(Markdown("### X validation:"))
display(X_val)

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [17]:
# Interpolate noncategorical missing values with median
import sklearn.impute
from sklearn.base import BaseEstimator, TransformerMixin

class ImputeNumerical(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.numerical_columns = []
    self.imp_mean = {}

  # def fit(self, X: pd.DataFrame, y):
  #   binary_features = ['is_sm_ips_ports', 'is_ftp_login']
  #   categorical_values = X.select_dtypes(include=['object'])
  #   binary_values = X.loc[:, binary_features]
  #   numerical_values = X.select_dtypes(include=['int64', 'float64']) \
  #                               .drop(binary_features, axis=1)
  #   self.numerical_columns = numerical_values.columns
  #   for col in self.numerical_columns:
  #     self.imp_mean[col] = sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='median')
  #     self.imp_mean[col].fit(numerical_values.loc[:, [col]])
  #   return self

  # def transform(self, X: pd.DataFrame):
  #   transformed_X = X.copy()
  #   for col in self.numerical_columns:
  #     if col in transformed_X.columns:
  #       transformed_X.loc[:, [col]] = self.imp_mean[col].transform(transformed_X.loc[:, [col]])
  #   return transformed_X

  def fit(self, X: pd.DataFrame, y):
    self.imp_mean = sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    self.imp_mean.fit(X)
    return self
  
  def transform(self, X: pd.DataFrame):
    return pd.DataFrame(self.imp_mean.transform(X), columns=X.columns)
  
class ImputeCategorical(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.categorical_columns = []
    self.imp_mode = {}

  # def fit(self, X: pd.DataFrame, y):
  #   binary_features = ['is_sm_ips_ports', 'is_ftp_login']
  #   categorical_values = X.select_dtypes(include=['object'])
  #   binary_values = X.loc[:, binary_features]
  #   numerical_values = X.select_dtypes(include=['int64', 'float64']) \
  #                               .drop(binary_features, axis=1)
  #   self.categorical_columns = categorical_values.columns
  #   for col in self.categorical_columns:
  #     self.imp_mode[col] = sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='most_frequent')
  #     self.imp_mode[col].fit(categorical_values.loc[:, [col]])
  #   return self

  # def transform(self, X: pd.DataFrame):
  #   transformed_X = X.copy()
  #   for col in self.categorical_columns:
  #     if col in transformed_X.columns:
  #       transformed_X.loc[:, [col]] = self.imp_mode[col].transform(transformed_X.loc[:, [col]])
  #   return transformed_X

  def fit(self, X: pd.DataFrame, y):
    self.imp_mode = sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    self.imp_mode.fit(X)
    return self
  
  def transform(self, X: pd.DataFrame):
    return pd.DataFrame(self.imp_mode.transform(X), columns=X.columns)

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd


class ReplaceOutliersWithMedian(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=3):
        self.threshold = threshold
        self.medians_ = None
    
    def fit(self, X, y=None):
        X = X.toarray() if hasattr(X, 'toarray') else np.array(X)

        self.medians_ = np.median(X, axis=0)
        
        return self
    
    def transform(self, X):
        if self.medians_ is None:
            raise ValueError("Transformer has not been fitted yet. Call 'fit' first.")
        
        X = X.toarray() if hasattr(X, 'toarray') else np.array(X)
        
        X_transformed = X.copy()
        
        outlier_mask = np.abs(X_transformed) > self.threshold
        X_transformed[outlier_mask] = self.medians_[np.where(outlier_mask)[1]]
        
        return X_transformed


class DropOutliers(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=3):
        self.threshold = threshold
        self.medians_ = None
    
    def fit(self, X, y=None):
        X = X.toarray() if hasattr(X, 'toarray') else np.array(X)

        self.medians_ = np.median(X, axis=0)
        
        return self
    
    def transform(self, X):
        if self.medians_ is None:
            raise ValueError("Transformer has not been fitted yet. Call 'fit' first.")
        
        X = X.toarray() if hasattr(X, 'toarray') else np.array(X)
        
        outlier_mask = np.abs(X) > self.threshold
        X_transformed = X[~np.any(outlier_mask, axis=1)]
        
        return X_transformed

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class RemoveDuplicates(BaseEstimator, TransformerMixin):
    def fit(self, X: pd.DataFrame, y):
        return self

    def transform(self, X: pd.DataFrame):
        return X.drop_duplicates(keep='first').reset_index(drop=True)

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [20]:
# Write your code here
from sklearn.feature_selection import SelectKBest, f_classif

class FilterSelection(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X: pd.DataFrame, y):
        self.select_k_best = SelectKBest(score_func=f_classif, k=10)
        self.select_k_best.fit(X, y)
        return self

    def transform(self, X: pd.DataFrame):
        selected_features = self.select_k_best.get_support()
        return X.loc[:, selected_features]

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [21]:
# Write your code here
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

class MinMaxFeatureScaling(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        X_scaled = X.copy()
        X_scaled = pd.DataFrame(self.scaler.transform(X), columns=X.columns)
        return X_scaled
    
class MaxAbsFeatureScaling(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MaxAbsScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        X_scaled = X.copy()
        X_scaled = pd.DataFrame(self.scaler.transform(X), columns=X.columns)
        return X_scaled

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [22]:
import sklearn.preprocessing
from sklearn.preprocessing import TargetEncoder

class OneHotCategorical(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.categorical_columns = []
    self.onehot = None

  def fit(self, X: pd.DataFrame, y):
    # binary_features = ['is_sm_ips_ports', 'is_ftp_login']
    # categorical_values = X.select_dtypes(include=['object'])
    # binary_values = X.loc[:, binary_features]
    # numerical_values = X.select_dtypes(include=['int64', 'float64']) \
    #                             .drop(binary_features, axis=1).drop(['id'], axis=1)
    # self.categorical_columns = categorical_values.columns
    self.onehot = sklearn.preprocessing.OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    self.onehot.fit(X)
    return self
  
  def transform(self, X: pd.DataFrame):
    # transformed_X = X.copy()
    # onehot_X = self.onehot.transform(transformed_X.loc[:, self.categorical_columns])
    # onehot_df = pd.DataFrame(onehot_X, columns=self.onehot.get_feature_names_out(self.categorical_columns), index=transformed_X.index)
    # to_return = pd.concat([transformed_X.drop(columns=self.categorical_columns), onehot_df], axis=1)
    # return to_return
    return pd.DataFrame(self.onehot.transform(X), columns=self.onehot.get_feature_names_out(), index=X.index)

class TargetCategorical(BaseEstimator, TransformerMixin):
  def __init__(self):
    self.categorical_columns = []
    self.target = None

  def fit(self, X: pd.DataFrame, y):
    # binary_features = ['is_sm_ips_ports', 'is_ftp_login']
    # categorical_values = X.select_dtypes(include=['object'])
    # binary_values = X.loc[:, binary_features]
    # numerical_values = X.select_dtypes(include=['int64', 'float64']) \
    #                             .drop(binary_features, axis=1).drop(['id'], axis=1)
    # self.categorical_columns = categorical_values.columns
    self.target = TargetEncoder(smooth="auto")
    self.target.fit(X, y)
    return self
  
  def transform(self, X: pd.DataFrame):
    # transformed_X = X.copy()
    # target_X = self.target.transform(transformed_X.loc[:, self.categorical_columns])
    # target_df = pd.DataFrame(target_X, columns=self.categorical_columns, index=transformed_X.index)
    # to_return = pd.concat([transformed_X.drop(columns=self.categorical_columns), target_df], axis=1)
    # return to_return
    return pd.DataFrame(self.target.transform(X), columns=self.target.get_feature_names_out(), index=X.index)

In [ ]:
%pip install imblearn

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [24]:
from imblearn.over_sampling import SMOTE

class SMOTEImbalance(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.smote = None

    def fit(self, X: pd.DataFrame, y: pd.DataFrame):
        self.smote = SMOTE(sampling_strategy='auto', random_state=42)
        self.smote.fit_resample(X, y) 
        return self

    def transform(self, X: pd.DataFrame, y: pd.DataFrame):
        X_resampled, y_resampled = self.smote.fit_resample(X, y)
        return pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=y.columns)

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class StandardScaleNumerical(BaseEstimator, TransformerMixin):
  def __init__(self, with_mean=True):
    self.with_mean = with_mean
    self.scaler = None

  def fit(self, X: pd.DataFrame, y):
    # binary_features = ['is_sm_ips_ports', 'is_ftp_login']
    # categorical_values = X.select_dtypes(include=['object'])
    # binary_values = X.loc[:, binary_features]
    # numerical_values = X.select_dtypes(include=['int64', 'float64']) \
    #                             .drop(binary_features, axis=1)

    self.scaler = StandardScaler(with_mean=self.with_mean)
    self.scaler.fit(X)

    return self

  def transform(self, X: pd.DataFrame):
    # binary_features = ['is_sm_ips_ports', 'is_ftp_login']
    # categorical_values = X.select_dtypes(include=['object'])
    # binary_values = X.loc[:, binary_features]
    # numerical_values = X.select_dtypes(include=['int64', 'float64']) \
    #                             .drop(binary_features, axis=1)
    
    # transform_X = self.scaler.transform(numerical_values)
    # scaled = pd.DataFrame(transform_X, columns=self.scaler.get_feature_names_out())
    
    # binary_values.reset_index(drop=True, inplace=True)
    # scaled.reset_index(drop=True, inplace=True)
    # categorical_values.reset_index(drop=True, inplace=True)
    
    # to_return = pd.concat([scaled, binary_values, categorical_values], axis=1)
    # return to_return
    return pd.DataFrame(self.scaler.transform(X), columns=self.scaler.get_feature_names_out())

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [26]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

numerical_pipeline = Pipeline([
    ('imputer', ImputeNumerical()),
    ('scaler', MinMaxFeatureScaling()),
    # ('standard', StandardScaleNumerical())
])

categorical_pipeline = Pipeline([
    ('imputer', ImputeCategorical()),
    ('onehot', OneHotCategorical()),
    ('scaler', MaxAbsFeatureScaling()),
    # ('standard', StandardScaleNumerical(with_mean=False))
])

preprocessor = ColumnTransformer([
    ('numerical', numerical_pipeline, [*numerical_features, *binary_features]),
    ('categorical', categorical_pipeline, [feature for feature in categorical_features if (feature != 'attack_cat')]),
])

pipe = Pipeline([
    ('preprocessor', preprocessor),
    # ('outlier', ReplaceOutliersWithMedian()), 
    ('filter', SelectKBest(score_func=f_classif, k=70)),
    # ('pca', PCA(n_components=70)),
])

In [ ]:
# # Your code should work up until this point
train_set = pipe.fit_transform(X_train, y_train)
val_set = pipe.transform(X_val)
print(train_set.shape)
print(val_set.shape)

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

In [29]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)

## A. KNN

In [ ]:
# Type your code here
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(train_set, y_train)

scores = cross_val_score(knn, train_set, y_train, cv=kf, scoring='accuracy')
print(f"Cross validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation of accuracy: {scores.std()}")

y_pred = knn.predict(val_set)
f1 = f1_score(y_val, y_pred, average='macro')
print(f"F1 score: {f1}")

df = pd.DataFrame()
df['attack_cat'] = y_val
df['predicted'] = y_pred
df.head()

## B. Naive Bayes

In [ ]:
# Type your code here
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(train_set, y_train)

scores = cross_val_score(nb, train_set, y_train, cv=kf, scoring='accuracy')
print(f"Cross validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation of accuracy: {scores.std()}")

y_pred = nb.predict(val_set)
f1 = f1_score(y_val, y_pred, average='macro')
print(f"F1 score: {f1}")

df = pd.DataFrame()
df['attack_cat'] = y_val
df['predicted'] = y_pred
df.head()

### Custom Naive Bayes

In [46]:
from sklearn.base import BaseEstimator, ClassifierMixin

class CustomNaiveBayes(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.classes = None
        self.class_priors = None
        self.class_means = None
        self.class_variances = None
    
    def fit(self, X, y):

        self.classes = np.unique(y)
        n_classes = len(self.classes)
        n_features = X.shape[1]

        self.class_means = np.zeros((n_classes, n_features))
        self.class_variances = np.zeros((n_classes, n_features))
        self.class_priors = np.zeros(n_classes)

        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.class_priors[i] = X_c.shape[0] / X.shape[0]
            self.class_means[i, :] = X_c.mean(axis=0)
            self.class_variances[i, :] = X_c.var(axis=0) + 1e-7
        
        return self
    
    def _gaussian_probability(self, x, mean, variance):
        exponent = np.exp(-((x - mean)**2 / (2 * variance)))
        return (1 / (np.sqrt(2 * np.pi * variance))) * exponent
    
    def predict_proba(self, X):
        n_samples = X.shape[0]
        n_classes = len(self.classes)
        
        probabilities = np.zeros((n_samples, n_classes))
        for i in range(n_classes):
            prior = np.log(self.class_priors[i])
            conditional = np.sum(np.log(
                self._gaussian_probability(
                    X, 
                    self.class_means[i, :], 
                    self.class_variances[i, :]
                ) + 1e-10 
            ), axis=1)
            
            
            probabilities[:, i] = prior + conditional
        
        probabilities = np.exp(probabilities)
        probabilities /= probabilities.sum(axis=1, keepdims=True)
        
        return probabilities
    
    def predict(self, X):
        probabilities = self.predict_proba(X)
        return self.classes[np.argmax(probabilities, axis=1)]
    
    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)


In [ ]:
model = CustomNaiveBayes()
model.fit(train_set, y_train)

scores = cross_val_score(model, train_set, y_train, cv=kf, scoring='accuracy')
print(f"Cross validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation of accuracy: {scores.std()}")

y_pred = model.predict(val_set)

f1 = f1_score(y_val, y_pred, average='macro')
print(f"F1 score: {f1}")

df = pd.DataFrame({'attack_cat': y_val, 'predicted': y_pred})
df.head()

## C. ID3

In [ ]:
# Type your code here
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

dt = DecisionTreeClassifier(criterion='entropy', random_state=42)
dt.fit(train_set, y_train)

scores = cross_val_score(dt, train_set, y_train, cv=kf, scoring='accuracy')
print(f"Cross validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation of accuracy: {scores.std()}")

y_pred = dt.predict(val_set)
f1 = f1_score(y_val, y_pred, average='macro')
print(f"F1 score: {f1}")

df = pd.DataFrame()
df['attack_cat'] = y_val
df['predicted'] = y_pred
df.head()

In [35]:
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import LabelEncoder

class ID3(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=5, n_bins=10):
        self.max_depth = max_depth
        self.n_bins = n_bins
    
    def _discretize(self, X):
        X_binned = np.zeros_like(X, dtype=int)
        for i in range(X.shape[1]):
            bins = np.linspace(X[:, i].min(), X[:, i].max(), self.n_bins + 1)
            X_binned[:, i] = np.digitize(X[:, i], bins[1:-1]) 
        return X_binned
    
    def fit(self, X, y):
        X = np.asarray(X, dtype=np.float64)
        le = LabelEncoder()
        y = le.fit_transform(y)
        X, y = check_X_y(X, y, dtype=np.float64)
        
        X_binned = self._discretize(X)
        
        self.tree_ = self._build_tree(X_binned, y)
        
        self.n_features_in_ = X.shape[1]
        self.classes_ = le.classes_
        self.label_encoder_ = le
        
        return self
    
    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))
    
    def _information_gain(self, X, y, feature_idx):
        total_entropy = self._entropy(y)
        
        unique_values = np.unique(X[:, feature_idx])
        
        weighted_entropies = np.array([
            len(y[X[:, feature_idx] == value]) / len(y) * 
            self._entropy(y[X[:, feature_idx] == value])
            for value in unique_values
        ])
        
        return total_entropy - np.sum(weighted_entropies)
    
    def _build_tree(self, X, y, depth=0):
        unique_classes = np.unique(y)
        
        if (len(unique_classes) == 1 or 
            depth == self.max_depth or 
            X.shape[1] == 0):
            return np.argmax(np.bincount(y)) if len(y) > 0 else None
        
        gains = np.array([
            self._information_gain(X, y, i) 
            for i in range(X.shape[1])
        ])
        
        best_feature = np.argmax(gains)
        max_gain = gains[best_feature]
        
        if max_gain == 0:
            return np.argmax(np.bincount(y))
        
        node = {'feature': best_feature}
        node['children'] = {}
        
        unique_values = np.unique(X[:, best_feature])
        
        for value in unique_values:
            mask = X[:, best_feature] == value
            
            X_subset = np.delete(X[mask], best_feature, axis=1)
            y_subset = y[mask]
            
            if len(y_subset) == 0:
                continue
            
            subtree = self._build_tree(
                X_subset,
                y_subset,
                depth=depth+1
            )
            
            node['children'][value] = subtree
        
        return node
    
    def predict(self, X):
        X = check_array(X, dtype=np.float64)
        check_is_fitted(self, ['tree_', 'classes_'])
        
        X_binned = self._discretize(X)
        
        predictions = np.array([self._predict_single(x) for x in X_binned])
        return self.label_encoder_.inverse_transform(predictions)
    
    def _predict_single(self, x):
        node = self.tree_
        
        while isinstance(node, dict):
            feature = node['feature']
            value = x[feature]
            
            node = node['children'].get(value, list(node['children'].values())[0])
            x = np.delete(x, feature)
        
        return node
    
    def predict_proba(self, X):
        X = check_array(X, dtype=np.float64)
        check_is_fitted(self, ['tree_', 'classes_'])
        
        predictions = self.predict(X)
        
        proba = np.zeros((X.shape[0], len(self.classes_)))
        proba[np.arange(len(predictions)), 
              self.label_encoder_.transform(predictions)] = 1
        
        return proba
    
    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)

In [ ]:
id3 = ID3(max_depth=70, n_bins=66)
id3.fit(train_set, y_train)

scores = cross_val_score(id3, train_set, y_train, cv=kf, scoring='accuracy', n_jobs=10, verbose=1)
print(f"Cross validation scores: {scores}")
print(f"Mean accuracy: {scores.mean()}")
print(f"Standard deviation of accuracy: {scores.std()}")

y_pred = id3.predict(val_set)
f1 = f1_score(y_val, y_pred, average='macro')
print(f"F1 score: {f1}")

df = pd.DataFrame()
df['attack_cat'] = y_val
df['predicted'] = y_pred
df.head()

## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [37]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [ ]:
# Membaca semua file csv test
additional_features_df = pd.read_csv('../dataset/test/additional_features_test.csv')
basic_features_df = pd.read_csv('../dataset/test/basic_features_test.csv')
content_features_df = pd.read_csv('../dataset/test/content_features_test.csv')
flow_features_df = pd.read_csv('../dataset/test/flow_features_test.csv')
time_features_df = pd.read_csv('../dataset/test/time_features_test.csv')

# Menggabungkan data training dan testing untuk analisis EDA menyeluruh
test_data = pd.merge(basic_features_df, additional_features_df, on="id")
test_data = pd.merge(test_data, content_features_df, on="id")
test_data = pd.merge(test_data, flow_features_df, on="id")
test_data = pd.merge(test_data, time_features_df, on="id")

data.head()

In [51]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

numerical_pipeline = Pipeline([
    ('imputer', ImputeNumerical()),
    ('scaler', MinMaxFeatureScaling()),
    # ('standard', StandardScaleNumerical())
])

categorical_pipeline = Pipeline([
    ('imputer', ImputeCategorical()),
    ('onehot', OneHotCategorical()),
    ('scaler', MaxAbsFeatureScaling()),
    # ('standard', StandardScaleNumerical(with_mean=False))
])

preprocessor = ColumnTransformer([
    ('numerical', numerical_pipeline, [*numerical_features, *binary_features]),
    ('categorical', categorical_pipeline, [feature for feature in categorical_features if (feature != 'attack_cat')]),
])

pipe = Pipeline([
    ('preprocessor', preprocessor),
    # ('outlier', ReplaceOutliersWithMedian()), 
    ('filter', SelectKBest(score_func=f_classif, k=70)),
    # ('pca', PCA(n_components=70)),
])

In [ ]:
train_set = pipe.fit_transform(X, y)
test_set = pipe.transform(test_data)
print(train_set.shape)
print(test_set.shape)

In [ ]:
model.fit(train_set, y)
with open('../model/nb.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

y_pred = model.predict(test_set)

df = pd.DataFrame()
df['id'] = test_data['id']
df['attack_cat'] = y_pred
df.to_csv('../submission/nb.csv', index=False)
df.head()

In [ ]:
id3.fit(train_set, y)
with open('../model/id3.pkl', 'wb') as model_file:
    pickle.dump(id3, model_file)

y_pred_id3 = id3.predict(test_set)

df = pd.DataFrame()
df['id'] = test_data['id']
df['attack_cat'] = y_pred_id3
df.to_csv('../submission/id3_scratch_submission.csv', index=False)
df.head()

In [ ]:
nb_scratch = CustomNaiveBayes()
nb_scratch.fit(train_set, y)

y_pred_nb_scratch = nb_scratch.predict(test_set)

df = pd.DataFrame()
df['id'] = test_data['id']
df['attack_cat'] = y_pred_nb_scratch
df.to_csv('../submission/nb_scratch_submission.csv', index=False)
df.head()

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`